In [94]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import pathlib
import os

import plotly.graph_objects as go
import ipywidgets as widgets

In [95]:
DATA_PATH = "../data"

In [96]:
files = os.listdir(DATA_PATH)
files = list(filter(lambda x: x.endswith(".pkl"), files))
files = list(filter(lambda x: x.startswith("henon_tunes"), files))

# get numbers from files
numbers = list(map(lambda x: float((x.split("_")[-1])[:-4]), files))

# make a dictionary
d = dict(zip(numbers, [os.path.join(DATA_PATH, f) for f in files]))

In [97]:
files = os.listdir(DATA_PATH)
files = list(filter(lambda x: x.endswith(".pkl"), files))
files = list(filter(lambda x: x.startswith("henon_long_tracking"), files))

# get numbers from files
numbers = list(map(lambda x: float((x.split("_")[-1])[:-4]), files))

# make a dictionary
d_track = dict(zip(numbers, [os.path.join(DATA_PATH, f) for f in files]))

In [ ]:
epsilon = 16.0

In [ ]:
with open(d[epsilon], "rb") as f:
    data = pickle.load(f)
with open(d_track[epsilon], "rb") as f:
    data_track = pickle.load(f)

In [118]:
def format_track_data(data, log10=True, clean=True):
    track = data["steps"].reshape(
        data["config"]["samples"], data["config"]["samples"])
    if log10:
        track = np.log10(track)
    if clean:
        track[np.isnan(track)] = np.nanmax(track)
        track[np.isinf(track)] = np.nanmax(track)
    return track
    

def format_data(data, power, element, log10=True, clean=True):
    settings = data["settings"]
    samples = settings["samples"]
    if element == "birkhoff":
        dataframe = data["birkhoff_tunes"]
    elif element == "fft":
        dataframe = data["fft_tunes"]
    else:
        raise ValueError("element must be either 'birkhoff' or 'fft'")

    power_middle = 2**power
    power_to = 2**(power+1)

    dataframe_middle_x = dataframe[(dataframe["from"] == 0) & (dataframe["to"] == power_middle)]["tune_x"].array[0].reshape(samples, samples)
    dataframe_middle_y = dataframe[(dataframe["from"] == 0) & (dataframe["to"] == power_middle)]["tune_y"].array[0].reshape(samples, samples)

    dataframe_to_x = dataframe[(dataframe["from"] == power_middle) & (dataframe["to"] == power_to)]["tune_x"].array[0].reshape(samples, samples)
    dataframe_to_y = dataframe[(dataframe["from"] == power_middle) & (dataframe["to"] == power_to)]["tune_y"].array[0].reshape(samples, samples)

    diff_x = dataframe_to_x - dataframe_middle_x
    diff_y = dataframe_to_y - dataframe_middle_y

    diff = np.sqrt(diff_x**2 + diff_y**2)
    
    if log10:
        diff = np.log10(diff)
    
    if clean:
        diff = np.asarray(diff, dtype=object)
        diff[np.isnan(diff.astype(np.float_))] = None
        diff[np.isinf(diff.astype(np.float_))] = None

    return diff


In [140]:
power_of_two = widgets.IntSlider(
    value=7,
    min=data["settings"]["min_power_of_two"],
    max=data["settings"]["max_power_of_two"],
    step=1,
    description="Power of 2:",
    continuous_update=False,
)

text_power_of_two = widgets.Label(
    value="N turns = {}".format(2 ** power_of_two.value),
)

d_keys = list(sorted(d.keys()))

epsilon_dropdown = widgets.Dropdown(
    options=d_keys,
    value=epsilon,
    description="Epsilon:",
    disabled=False,
)

# create stability heatmap
stab_plot = go.FigureWidget(
    data=go.Heatmap(z=np.log10(
        data_track["steps"].reshape(data_track["config"]["samples"], data_track["config"]["samples"])), colorscale="Viridis")
)
stab_plot.update_layout(
    title_text="Stability plot [log10], epsilon = {}".format(epsilon_dropdown.value),
    xaxis_title="x", yaxis_title="y"
)


# create heatmap with plotly
w1 = go.FigureWidget(data=go.Heatmap(
    z=format_data(data, power_of_two.value, "fft"), colorscale="Viridis"),
)
w1.update_layout(
    title_text="FFT tune [log10], epsilon = {}".format(epsilon_dropdown.value),
    xaxis_title="x", yaxis_title="y"
)
w2 = go.FigureWidget(data=go.Heatmap(
    z=format_data(data, power_of_two.value, "birkhoff"), colorscale="Viridis"),
)
w2.update_layout(
    title_text="Birkhoff tune [log10], epsilon = {}".format(
        epsilon_dropdown.value),
    xaxis_title="x", yaxis_title="y"
)
img_container = widgets.VBox([
    widgets.HBox([power_of_two, text_power_of_two, epsilon_dropdown], width='auto'),
    widgets.HBox([w1, w2], width='auto'),
    widgets.HBox([stab_plot], width='auto')
])


/tmp/ipykernel_124973/2896831784.py:37: RuntimeWarning:

divide by zero encountered in log10



In [141]:
def response(change):
    epsilon = epsilon_dropdown.value
    power = power_of_two.value
    text_power_of_two.value = "N turns = {}".format(2 ** power_of_two.value)
    with open(d[epsilon], "rb") as f:
        data = pickle.load(f)
    with open(d_track[epsilon], "rb") as f:
        data_track = pickle.load(f)
    stab_plot.data[0].z = z = np.log10(
        data_track["steps"].reshape(data_track["config"]["samples"], data_track["config"]["samples"]))
    stab_plot.update_layout(
        title_text="Stability plot [log10], epsilon = {}".format(epsilon_dropdown.value))
    w1.data[0].z = format_data(data, power, "fft")
    w2.data[0].z = format_data(data, power, "birkhoff")
    w2.update_layout(
        title_text="FFT Tunes [log10], epsilon = {}".format(epsilon))
    w1.update_layout(
        title_text="Birkhoff Tunes [log10], epsilon = {}".format(epsilon))

power_of_two.observe(response, names="value")
epsilon_dropdown.observe(response, names="value")


In [142]:
img_container

In [112]:
epsilon = 16.0

In [113]:
with open(d[epsilon], "rb") as f:
    data = pickle.load(f)
with open(d_track[epsilon], "rb") as f:
    data_track = pickle.load(f)


In [ ]:
format_data(data, power_of_two.value, "fft")